This notebook compares different MPCs for the differential-drive robot state-tracking task.

To load cost and timing results from the paper, run the setup and directly load the results (Skip II) and V)

To rerun the experiments on your machine and save your own results file, change the result file names in III) and VI) and run all cells of the notebook.

I) Setup

In [ ]:

from differential_drive.Differential_drive_MPC import DifferentialDriveMPC, DifferentialDriveMPCOptions
import gc
import shutil
import numpy as np
import matplotlib.pyplot as plt
from differential_drive.utils_diff_drive import simulate_closed_loop
from utils_shared import compute_exponential_step_sizes
from differential_drive.plotting_utils_diff_drive import plot_diff_drive_trajectory

In [ ]:
# initialize sim solver for closed loop simulation, we also use the DifferentialDrive MPC class to instatiate the sim solver.
mpc_opts_sim = DifferentialDriveMPCOptions()
mpc_opts_sim.N = 1
dt_sim = 0.005
mpc_opts_sim.step_sizes = [dt_sim]*1
mpc_opts_sim.switch_stage = 2
mpc_sim = DifferentialDriveMPC(mpc_opts_sim)
sim_solver = mpc_sim.acados_sim_solver

# MPC config shared by all MPCs 
dt_inital_mpc = 0.01
control_step = int(dt_inital_mpc/dt_sim) # determines zero order hold of the MPC
X0 = np.array([-1.0, 1.0, 0.0, -np.pi/2, 0.0, 0.0, 0.0])
duration = 15.0 # was 15.0

# collect mean costs and solve times
mean_costs = []
mean_solve_times = []
mean_solve_time_per_iter = []

# collect standart deviations
std_dev_costs = []
std_dev_solve_times = []
std_dev_solve_time_per_iter = []

del mpc_sim
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

# for plotting
initial_state_plotting = np.vstack([X0[0], X0[1], X0[3]])
y_ref_plotting = np.zeros(3)
latexify=True
save=True

II) Comparison of different predictive controllers

As ground truth model we consider the model integrated with step size 0.005. Consistently, we simulate at 200Hz and apply the MPCs with a frequency (zero order hold) corresponding to their initial step size.

For our experiment, we compare the following MPCs. 

0.) Baseline: Long Horizon MPC, step size is 0.01 and we have a long horizon length of 1000. Thus, the MPC's planning horizon captures half of the entire 20s simulation. The MPC reaches a state very close to the goal state and achieves small costs. 

1.) Myopic MPC: Same as 0) but with a shorter planning horizon. 

2.) Larger integration step sizes and a corresponding lower frequency.

4.) Full-model MPC with exponential increase of step size at the same rate as 6). The time horizon is still 20s.

5.) Model-switching: Same as 0), but model is switched roughly at the same switching time as 6)

6.) MTS-MPC: Combining exponential increase of step size with model switching. Additionally the Integrator is switched to ERK in the second phase.

-1.) To show that implicite integration is neccessary: exactly as 0), but integrated with ERK. We get solver errors.

In [ ]:
### -1) ERK usage (to show that IRK is necessary) ###
mpc_opts_ERK = DifferentialDriveMPCOptions()
N = 1000
mpc_opts_ERK.N = N
mpc_opts_ERK.step_sizes = [dt_inital_mpc]*N
mpc_opts_ERK.switch_stage = N+1
mpc_opts_ERK.integrator_type = "ERK"
mpc_ERK = DifferentialDriveMPC(mpc_opts_ERK)

x_traj_ERK, u_traj_ERK, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_ERK, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_ERK, closed_loop_traj=x_traj_ERK, latexify=latexify, number=None, legend=False, save=False, initial_state=initial_state_plotting)
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]

del mpc_ERK
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 0) Baseline ###
mpc_opts_baseline = DifferentialDriveMPCOptions()
N = 1000
mpc_opts_baseline.N = N
mpc_opts_baseline.step_sizes = [dt_inital_mpc]*N
mpc_opts_baseline.switch_stage = N+1
mpc_opts_baseline.integrator_type = "IRK"
mpc_baseline = DifferentialDriveMPC(mpc_opts_baseline)

x_traj_0, u_traj_0, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_baseline, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_baseline, closed_loop_traj=x_traj_0, latexify=latexify, number=0, legend=False, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_baseline
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 1) Myopic Exact model MPC ###
mpc_opts_1 = DifferentialDriveMPCOptions()
N = 250
mpc_opts_1.N = N
mpc_opts_1.step_sizes = [dt_inital_mpc]*N
mpc_opts_1.switch_stage = N+1
mpc_opts_1.integrator_type = "IRK"
mpc_1 = DifferentialDriveMPC(mpc_opts_1)

x_traj_1, u_traj_1, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_1, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_1, closed_loop_traj=x_traj_1, latexify=latexify, number=1, legend=True, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_1
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 2) Larger step size ###
mpc_opts_2 = DifferentialDriveMPCOptions()
N = 250
mpc_opts_2.N = N
mpc_opts_2.step_sizes = [4*dt_inital_mpc]*N
mpc_opts_2.switch_stage = N+1
mpc_opts_2.integrator_type = "IRK"
mpc_2 = DifferentialDriveMPC(mpc_opts_2)

x_traj_2, u_traj_2, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_2, duration, sim_solver=sim_solver, control_step=4*control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_2, closed_loop_traj=x_traj_2, latexify=latexify, number=2, legend=False, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_2
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 4) Exponential increase in stepsize ###
mpc_opts_4 = DifferentialDriveMPCOptions()
N = 80
mpc_opts_4.N = N
mpc_opts_4.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=10,
    N_steps=N,
    plot=False
)
mpc_opts_4.switch_stage = N+1
mpc_opts_4.integrator_type = "IRK"
mpc_4 = DifferentialDriveMPC(mpc_opts_4)

x_traj_4, u_traj_4, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_4, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_4, closed_loop_traj=x_traj_4, latexify=latexify, number=4, legend=False, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_4
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

print(np.mean(stage_costs))

In [ ]:
### 5) Model switching ###
mpc_opts_5 = DifferentialDriveMPCOptions()
N = 1000
mpc_opts_5.N = N
mpc_opts_5.step_sizes = [dt_inital_mpc]*N
mpc_opts_5.switch_stage = 36
mpc_opts_5.integrator_type = "IRK"
mpc_5 = DifferentialDriveMPC(mpc_opts_5)

x_traj_5, u_traj_5, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_5, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_5, closed_loop_traj=x_traj_5, latexify=latexify, number=5, legend=False, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_5
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 6) Ours Mixed model, exponential step size increase ###
mpc_opts_6 = DifferentialDriveMPCOptions()
N = 80
mpc_opts_6.N = N
mpc_opts_6.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=10,
    N_steps=N,
    plot=False
)
print(sum(mpc_opts_6.step_sizes[0:20]))
mpc_opts_6.switch_stage = 21 # only keep actuator model in first part of the horizon
mpc_opts_6.integrator_type = "IRK" # switch to ERK happens automatically in the class
mpc_6 = DifferentialDriveMPC(mpc_opts_6)

x_traj_6, u_traj_6, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_6, duration, sim_solver=sim_solver, control_step=control_step)
plot_diff_drive_trajectory(y_ref_plotting, mpc=mpc_6, closed_loop_traj=x_traj_6, latexify=latexify, number=6, legend=False, save=save, initial_state=initial_state_plotting)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_6
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

print(np.mean(stage_costs))

III) Safe or load a results file 

In [ ]:
# save results or load them
import os
import pickle
from utils_shared import get_dir

data_dir = get_dir("data")
# If you want to run the experiments on your own machine and save the results, adjust the results file name
results_file = data_dir / "differential_drive/diff_drive_results.pkl" 
print(os.path.exists(results_file))

if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        data = pickle.load(f)
    mean_costs = data['mean_costs']
    mean_costs_baseline = mean_costs[0]
    mean_solve_times = data['mean_solve_times']
    mean_solve_time_per_iter = data['mean_solve_time_per_iter']
    x_traj_0 = data['x_traj_0']
    x_traj_6 = data['x_traj_6']
    std_dev_costs = data['std_dev_costs']
    std_dev_solve_times = data['std_dev_solve_times']
    std_dev_solve_time_per_iter = data['std_dev_solve_time_per_iter']
else:
    # Save the computed results
    data = {
        'mean_costs': mean_costs,
        'mean_solve_times': mean_solve_times,
        'mean_solve_time_per_iter': mean_solve_time_per_iter,
        'x_traj_0': x_traj_0,
        'x_traj_6': x_traj_6,
        'std_dev_costs': std_dev_costs,
        'std_dev_solve_times': std_dev_solve_times,
        'std_dev_solve_time_per_iter': std_dev_solve_time_per_iter
    }
    os.makedirs(os.path.dirname(results_file), exist_ok=True)
    with open(results_file, 'wb') as f:
        pickle.dump(data, f)


IV) Plot and print results

In [ ]:
# Visualize baseline vs MTS-MPC Trajectory
plot_diff_drive_trajectory(y_ref_plotting, mpc=None, closed_loop_traj=x_traj_6, latexify=latexify, number=None, legend=False, save=save, initial_state=initial_state_plotting, closed_loop_traj_baseline=x_traj_0)

In [ ]:
# Plotting options for the following plots
latexify=True
save=True

In [ ]:
print(mean_costs)
print(mean_solve_times)
print(mean_solve_time_per_iter)

print(std_dev_costs)
print(std_dev_solve_times)
print(std_dev_solve_time_per_iter)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import barplot

# Data: mean_costs, mean_solve_times, mean_solve_times_per_iter should be defined already
approach_labels = [
    '0) Baseline',
    '1) Shorter Horizon',
    '2) Larger Step Size',
    '4) Increasing\n    Step Sizes',
    '5) Model Switching',
    '6) Model Switching +\n    Increasing Step Sizes'
]


barplot(
    approach_labels,
    mean_costs,
    mean_solve_times,
    mean_solve_time_per_iter,
    subpath="differential_drive/diff_drive_barplot.pdf",
    latexify=latexify,
    save=save,
    figsize=(8, 10),
    fontsize=12,
)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import pareto_frontier

# distinct symbols per approach
markers = ['D', 's', '^', 'v', 'o', 'X', 'p']

# Marker sizes per approach (indexed same as markers)
marker_sizes = [500, 500, 500, 500, 500, 500, 500]  # customize these

pareto_frontier(
    mean_solve_times,
    mean_costs,
    mean_costs_baseline,
    approach_labels,
    markers,
    marker_sizes,
    subpath="differential_drive/pareto_front_diff_drive.pdf",
    latexify=latexify,
    save=save,
    figsize=(7.5, 6),
    fontsize=25,
    x_lim=0.009,
    x_lim_upper=0.3,
    y_nonlog=True,
    legend=False
)


V) Sweep over switching indices

In [ ]:
### sweep ###
mpc_opts_sweep = DifferentialDriveMPCOptions()
N = 80
mpc_opts_sweep.N = N
mpc_opts_sweep.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=10,
    N_steps=N,
    plot=False
)
mpc_opts_sweep.integrator_type = "IRK" # switch to ERK happens automatically in the class
switching_indices = list(range (1, 80))
mean_costs_sweep = []
for switch_index in switching_indices:
    mpc_opts_sweep.switch_stage = switch_index 
    mpc_sweep = DifferentialDriveMPC(mpc_opts_sweep)

    _, _, stage_costs, solve_times, SQP_iters = simulate_closed_loop(X0, mpc_sweep, duration, sim_solver=sim_solver, control_step=control_step)

    del mpc_sweep
    gc.collect()
    shutil.rmtree('c_generated_code', ignore_errors=True)

    # mean values
    mean_costs_sweep.append(np.mean(stage_costs))




VI) Safe or load sweep results and plot them

In [ ]:
from utils_shared import get_dir
import pickle
import os
data_dir = get_dir("data")
# If you want to run the experiments on your own machine and save the results, adjust the results file name
results_file_sweep = data_dir / "differential_drive/differential_drive_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep = data['mean_costs_sweep']
    switching_indices = data['switching_indices']
    mean_costs_baseline = data['mean_costs_baseline']
else:
    data = {
        'mean_costs_sweep': mean_costs_sweep,
        'switching_indices': switching_indices,
        'mean_costs_baseline': mean_costs_baseline,
    }
    os.makedirs(os.path.dirname(results_file_sweep), exist_ok=True)
    with open(results_file_sweep, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(8,5))
plt.plot(switching_indices[1:], 100*np.array((mean_costs_sweep[1:]-mean_costs_baseline))/mean_costs_baseline, marker='o', linestyle='-', linewidth=2)

plt.xlabel("Switching Index", fontsize=12)
plt.ylabel("Mean Closed Loop Cost Increase in \%", fontsize=12)
plt.title("Mean Closed Loop Costs vs. Switching Index", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
#plt.ylim(0, 10)
plt.yscale('log')
plt.show()